In [1]:
# pip install openpyxl pandas

In [2]:
import pandas as pd
df = pd.read_excel('Technical Assistance Registry 2025.xlsx', sheet_name='Sum up')
df.head()

,Year,Mth,Report number,Day,Week No.,Engine,Variant,S/n,Operator,Place,...,Unnamed: 24,Staff,status,Invoiced? Y/N,Invoice No.,Promised Date to the customer,No.,Noria Y/N,Warranty Y/N,Scheduled/Unscheduled/AOG
0,2013,MARCH,NaN,14,11,Accessories,NaN,45001,PT TRAVIRA,KOTA BHARU,...,NaN,ADELINE,NaN,NaN,IV-00335,NaN,NaN,N,N,Unscheduled
1,2013,MARCH,NaN,14,11,ARRIEL 2,2S2,42295TEC,PT TRAVIRA,KOTA BHARU,...,NaN,ADELINE,NaN,NaN,IV-00336,NaN,NaN,N,N,Unscheduled
2,2013,MARCH,NaN,27,13,ARRIEL 1,1S1,15552TEC,MHS,GTA,...,NaN,JEROME,NaN,NaN,IV-00359,NaN,NaN,Y,N,Scheduled
3,2013,APR,NaN,9,15,Accessories,2D,50114,PT AIRFAST,IRIAN JAYA,...,NaN,ADELINE,NaN,NaN,IV-00380,NaN,NaN,Y,N,Scheduled
4,2013,MAY,NaN,10,19,ARRIEL 1,1S1,15129,MHS,GTA,...,NaN,JEROME,NaN,NaN,IV-00412,NaN,NaN,N,N,Scheduled


In [3]:
df.isna().sum()

Year                                             0
Mth                                              1
Report number                                  133
Day                                             43
Week No.                                        49
Engine                                           0
Variant                                          5
S/n                                             14
Operator                                        35
Place                                           34
Country                                          0
Brief Description of Technical Intervention      4
Intervention level                               7
Location                                         1
By                                               5
Technical Intervention Report Ref:              28
Type of release                                224
EASA F1  Cert. No.                             126
EQE                                            166
Standard time                  

In [4]:
# Remove specified columns
columns_to_remove = [
    'Report number',
    'Week No.',
    'S/n',
    'Place',
    'Country',
    'Location',
    'Technical Intervention Report Ref:',
    'Type of release',
    'EASA F1  Cert. No.',
    'EQE',
    'Standard time',
    'TAT',
    'Delta',
    'Remark',
    'Work File Ref',
    'Unnamed: 24',
    'Staff',
    'status ',
    'Invoiced? Y/N',
    'Invoice No.',
    'Promised Date to the customer',
    'No.',
    'Noria Y/N',
    'Warranty Y/N',
    'Scheduled/Unscheduled/AOG'
]

# Drop columns that exist in the dataframe
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

print(f"Remaining columns after removal: {df.columns.tolist()}")
df.head()

Remaining columns after removal: ['Year', 'Mth', 'Day', 'Engine', 'Variant', 'Operator', 'Brief Description of Technical Intervention', 'Intervention level', 'By']


,Year,Mth,Day,Engine,Variant,Operator,Brief Description of Technical Intervention,Intervention level,By
0,2013,MARCH,14,Accessories,NaN,PT TRAVIRA,HMU,L2,BS
1,2013,MARCH,14,ARRIEL 2,2S2,PT TRAVIRA,MODULE 4 & 5 REPLACEMENT,L2,BS
2,2013,MARCH,27,ARRIEL 1,1S1,MHS,"TU347,MO5 REPLACEMENT",L3,BS
3,2013,APR,9,Accessories,2D,PT AIRFAST,HMU SLEEVE REPLACEMENT,L2,BS
4,2013,MAY,10,ARRIEL 1,1S1,MHS,MODULE 1 REPLACEMENT,L2,BS


In [5]:
df = df.dropna(subset=['Brief Description of Technical Intervention'])
df['Brief Description of Technical Intervention'].count()

np.int64(309)

In [6]:
df['Brief Description of Technical Intervention'].unique()

array(['HMU', 'MODULE 4 & 5 REPLACEMENT', 'TU347,MO5 REPLACEMENT',
       'HMU SLEEVE REPLACEMENT', 'MODULE 1 REPLACEMENT',
       'MODULE 5 REPLACEMENT', 'TU 166 APPLICATION', 'HMU TBO EXTENTION',
       'O RING REPLACEMENT MO5', 'TU166', 'MODULE 1 & 5 REPLACEMENT',
       'MODULE 3 REPLACEMENT', 'TU347', 'MODULE 2/3 REPLACEMENT',
       'MODULE 1/3 REPLACEMENT', 'POST RENTAL INSPECTION',
       'MODULE 4 REPLACEMENT', 'MODULE 1,4 AND 5 REPLACEMENT',
       'LIGHTNING STRIKE INSPECTION', 'MODULE 1 OIL DRIP',
       'NFT SEALOL SEAL REPLACEMENT', 'REPLACEMENT MODULE 1',
       'MODULE 1 INSPECTION', 'MO1 VIBRATION CHECK', 'MO2 REPLACEMENT ',
       'TU 347 & SELOL SEAL CHANGE', 'COPPER SEAL CHANGE',
       'HMU SN1424 APPL SB 292 73 2836', 'TU 347 + TU 360', 'TU 357',
       'MO2 Replaced ', 'S/E M01', 'S/E M01, 03', 'S/E M01, 02 and 03',
       'M02 REMOVAL', 'KANBAN REPORT', 'Engine inspection',
       'Bleed valve inspection', 'MSB 292 72 2861',
       'S/E MO2 & HMU installation', 

### Split multiple tasks

In [7]:
import re

def split_tasks(text):
    # First replace S/E with STANDARD EXCHANGE
    text = re.sub(r'\bS/E\b', 'STANDARD EXCHANGE', text)
    
    # Replace &, +, /, and " AND " with commas for consistent splitting
    text = text.replace('&', ', ').replace(' + ', ', ').replace('/', ', ')
    text = re.sub(r'\s+AND\s+', ', ', text)
    text = re.sub(r'\s+and\s+', ', ', text)
    
    # Split by comma
    tasks = [task.strip() for task in text.split(',') if task.strip()]
    return tasks

# Apply splitting
task_lists = df['Brief Description of Technical Intervention'].apply(split_tasks)

# Find max number of tasks
max_tasks = task_lists.apply(len).max()
print(f"Maximum number of tasks in one entry: {max_tasks}")

# Create Task columns
for i in range(max_tasks):
    df[f'Task_{i+1}'] = task_lists.apply(lambda x: x[i] if i < len(x) else None)

# Display results
df[['Brief Description of Technical Intervention', 'Task_1', 'Task_2', 'Task_3', 'Task_4']].head(30)

Maximum number of tasks in one entry: 4


,Brief Description of Technical Intervention,Task_1,Task_2,Task_3,Task_4
0,HMU,HMU,None,None,None
1,MODULE 4 & 5 REPLACEMENT,MODULE 4,5 REPLACEMENT,None,None
2,"TU347,MO5 REPLACEMENT",TU347,MO5 REPLACEMENT,None,None
3,HMU SLEEVE REPLACEMENT,HMU SLEEVE REPLACEMENT,None,None,None
4,MODULE 1 REPLACEMENT,MODULE 1 REPLACEMENT,None,None,None
5,MODULE 5 REPLACEMENT,MODULE 5 REPLACEMENT,None,None,None
6,TU 166 APPLICATION,TU 166 APPLICATION,None,None,None
7,TU 166 APPLICATION,TU 166 APPLICATION,None,None,None
8,HMU TBO EXTENTION,HMU TBO EXTENTION,None,None,None
9,O RING REPLACEMENT MO5,O RING REPLACEMENT MO5,None,None,None


In [8]:
df['Task_1'].unique()

array(['HMU', 'MODULE 4', 'TU347', 'HMU SLEEVE REPLACEMENT',
       'MODULE 1 REPLACEMENT', 'MODULE 5 REPLACEMENT',
       'TU 166 APPLICATION', 'HMU TBO EXTENTION',
       'O RING REPLACEMENT MO5', 'TU166', 'MODULE 1',
       'MODULE 3 REPLACEMENT', 'MODULE 2', 'POST RENTAL INSPECTION',
       'MODULE 4 REPLACEMENT', 'LIGHTNING STRIKE INSPECTION',
       'MODULE 1 OIL DRIP', 'NFT SEALOL SEAL REPLACEMENT',
       'REPLACEMENT MODULE 1', 'MODULE 1 INSPECTION',
       'MO1 VIBRATION CHECK', 'MO2 REPLACEMENT', 'TU 347',
       'COPPER SEAL CHANGE', 'HMU SN1424 APPL SB 292 73 2836', 'TU 357',
       'MO2 Replaced', 'STANDARD EXCHANGE M01', 'M02 REMOVAL',
       'KANBAN REPORT', 'Engine inspection', 'Bleed valve inspection',
       'MSB 292 72 2861', 'STANDARD EXCHANGE MO2', 'SUPERVISION',
       'TECHNICAL REPORT', 'Application TU 81 DECU SN 783',
       'Application TU 81 DECU SN 701', 'Application TU 81 DECU SN 560',
       'Application TU 81 DECU SN 609',
       'Application TU 81 SPARE

In [9]:
def standardize_task1(text):
    """
    Comprehensive standardization for Task_1 column
    """
    if pd.isna(text):
        return text
    
    text = text.upper()
    
    # Fix common typos first
    text = re.sub(r'\bREPALCEMENT\b', 'REPLACEMENT', text)
    # text = re.sub(r'\bREPALEMENT\b', 'REPLACEMENT', text)
    text = re.sub(r'\bCHK\b', 'CHECK', text)
    
    # Standardize all module references to MO format: MODULE/MOD/M01/M02/MO01/MO02 -> MO1/MO2
    text = re.sub(r'\bMODULE\s+0?(\d)\b', r'MO\1', text)  # MODULE 01 or MODULE 1 -> MO1
    text = re.sub(r'\bMOD\s+0?(\d)\b', r'MO\1', text)     # MOD 02 or MOD 2 -> MO2
    # text = re.sub(r'\bMO0(\d)\b', r'MO\1', text)          # MO02 -> MO2
    text = re.sub(r'\bM0(\d)\b', r'MO\1', text)           # M02 -> MO2
    # text = re.sub(r'\bM(\d)(?=\s|$)\b', r'MO\1', text)    # M2 -> MO2 (standalone only)
    
    # Standardize TU references - add space after TU
    text = re.sub(r'\bTU\s*(\d+)\b', lambda m: f'TU {m.group(1)}', text)  # TU166 or TU 166 -> TU 166

    # "TU 166 APPLICATION" -> "TU 166"
    text = re.sub(r'\bTU\s+(\d+)\s+APPLICATION\b', r'TU \1', text)

    # "TEST OF THE VISUAL BLOCKAGE INDICATOR" -> "VISUAL BLOCKAGE INDICATOR TEST"
    text = re.sub(r'\bTEST\s+OF\s+THE\s+VISUAL\s+BLOCKAGE\s+INDICATOR\b', 'VISUAL BLOCKAGE INDICATOR TEST', text)
    text = re.sub(r'\bTEST\s+OF\s+VISUAL\s+BLOCKAGE\s+INDICATOR\b', 'VISUAL BLOCKAGE INDICATOR TEST', text)
    
    # Standardize TF references - add space after TF
    text = re.sub(r'\bTF\s*(\d+)\b', lambda m: f'TF {m.group(1)}', text)  # TF91 or TF 91 -> TF 91
    
    # Inspection standardization
    text = re.sub(r'\bBOROSCOPE\b', 'BORESCOPE', text)
    text = re.sub(r'\bENGINE BORESCOPE INSPECTION\b', 'BORESCOPE INSPECTION', text)
    text = re.sub(r'^BORESCOPE$', 'BORESCOPE INSPECTION', text)  # BORESCOPE alone -> BORESCOPE INSPECTION
    
    # "INSPECTION OF MO2" -> "MO2 INSPECTION"
    text = re.sub(r'\bINSPECTION\s+OF\s+(MO\d+)\b', r'\1 INSPECTION', text)
    
    # KANBAN standardization
    text = re.sub(r'^KANBAN$', 'KANBAN REPORT', text)
    
    # Replacement patterns: "REPLACEMENT MO2" or "REPLACEMENT MODULE 1" -> "MO2 REPLACEMENT"
    text = re.sub(r'\bREPLACED\b', 'REPLACEMENT', text)  # MO2 REPLACED -> MO2 REPLACEMENT
    text = re.sub(r'\bREPLACEMENT\s+(MO\d+)\b', r'\1 REPLACEMENT', text)
    text = re.sub(r'\bREPLACEMENT\s+MODULE\s+0?(\d)\b', r'MO\1 REPLACEMENT', text)
    
    # Installation patterns: "INSTALLATION OF THE HMU" or "INSTALLATION HMU" or "INSTALLATION OF MODULE 1" -> "HMU INSTALLATION" or "MO1 INSTALLATION"
    text = re.sub(r'\bINSTALLATION\s+OF\s+THE\s+(HMU|FCU)\b', r'\1 INSTALLATION', text)
    text = re.sub(r'\bINSTALLATION\s+OF\s+(HMU|FCU)\b', r'\1 INSTALLATION', text)
    text = re.sub(r'\bINSTALLATION\s+(HMU|FCU)\b', r'\1 INSTALLATION', text)
    # handle variations where MODULE was already converted to MO or where there's no space between MO and number
    text = re.sub(r'\bINSTALLATION\s+OF\s+MODULE\s+0?(\d)\b', r'MO\1 INSTALLATION', text)  # direct MODULE pattern
    text = re.sub(r'\bINSTALLATION\s+OF\s+MO\s*0?(\d)\b', r'MO\1 INSTALLATION', text)      # INSTALLATION OF MO1 or MO 1
    text = re.sub(r'\bINSTALLATION\s+MO\s*0?(\d)\b', r'MO\1 INSTALLATION', text)           # INSTALLATION MO1
    
    # Add REPLACEMENT to standalone module numbers: "MO4" -> "MO4 REPLACEMENT"
    if re.match(r'^MO\d+$', text):
        text = text + ' REPLACEMENT'
    
    # S/E = STANDARD EXCHANGE
    # text = re.sub(r'\bS/E\b', 'STANDARD EXCHANGE', text)
    
    # Spelling standardization
    text = re.sub(r'\bSTANDART\b', 'STANDARD', text)
    text = re.sub(r'\bASSESMENT\b', 'ASSESSMENT', text)
    # text = re.sub(r'\bASSESSMENT\b', 'ASSESSMENT', text)

    # "ASSESSMENT OF ENGINE" -> "ENGINE ASSESSMENT"
    text = re.sub(r'\bASSESSMENT\s+OF\s+ENGINE\b', 'ENGINE ASSESSMENT', text)
    
    return text

# Apply standardization to Task_1
df['Task_1'] = df['Task_1'].apply(standardize_task1)

# Show unique values
print("Unique Task_1 values after comprehensive standardization:")
print(f"Total unique tasks: {df['Task_1'].nunique()}")
print("\nSorted list:")
for task in sorted(df['Task_1'].unique()):
    print(f"  - {task}")

Unique Task_1 values after comprehensive standardization:
Total unique tasks: 82

Sorted list:
  - 500 HRS INSPECTION
  - APPLICATION TU 81 DECU SN 512
  - APPLICATION TU 81 DECU SN 522
  - APPLICATION TU 81 DECU SN 543
  - APPLICATION TU 81 DECU SN 560
  - APPLICATION TU 81 DECU SN 609
  - APPLICATION TU 81 DECU SN 701
  - APPLICATION TU 81 DECU SN 732
  - APPLICATION TU 81 DECU SN 760
  - APPLICATION TU 81 DECU SN 765
  - APPLICATION TU 81 DECU SN 783
  - APPLICATION TU 81 DECU SN 789
  - APPLICATION TU 81 DECU SN 908
  - APPLICATION TU 81 DECU SN 929
  - APPLICATION TU 81 DECU SN 930
  - APPLICATION TU 81 SPARE DECU SN 784
  - ASSISTANCE
  - BLEED VALVE INSPECTION
  - BORESCOPE INSPECTION
  - CERTIFICATION OF F.C.U
  - COMPRESSOR COVER REPLACEMENT
  - COPPER SEAL CHANGE
  - ENGINE ASSESSMENT
  - ENGINE INSPECTION
  - HELICOIL
  - HMU
  - HMU INSTALLATION
  - HMU SLEEVE REPLACEMENT
  - HMU SN1424 APPL SB 292 73 2836
  - HMU TBO EXTENTION
  - INSTALLATION OF F.C.U
  - KANBAN REPORT
  

In [10]:
df['Task_2'].unique()

array([None, '5 REPLACEMENT', 'MO5 REPLACEMENT', '3 REPLACEMENT', '4',
       'SELOL SEAL CHANGE', 'TU 360', '03', '02', 'HMU installation',
       '150h', 'update cycles', 'MO3 REMOVAL', 'MO3 7 TU198',
       'REPLACEMENT front firewall module 01', 'TU 377', 'installation',
       'FCU REPLACEMENT'], dtype=object)

In [11]:
def standardize_task2(text):
    """
    Standardization rules specific for Task_2 column
    """
    if pd.isna(text):
        return text
    
    # Convert to uppercase first
    text = text.upper()
    
    # "5 REPLACEMENT" -> "MO5 REPLACEMENT"
    # "3 REPLACEMENT" -> "MO3 REPLACEMENT"
    text = re.sub(r'\b(\d)\s+REPLACEMENT\b', r'MO\1 REPLACEMENT', text)
    
    # Standalone single digit "4" -> "MO4 REPLACEMENT"
    if re.match(r'^\d$', text):
        text = f'MO{text} REPLACEMENT'
    
    # "03" or "02" alone -> "STANDARD EXCHANGE 03" or "STANDARD EXCHANGE 02"
    if re.match(r'^0\d$', text):
        text = f'STANDARD EXCHANGE {text}'
    
    # "150h", "500h", "600h" -> "150H INSPECTION", "500H INSPECTION", "600H INSPECTION"
    text = re.sub(r'\b(\d+)H\b', r'\1H INSPECTION', text)
    
    # Add space after TU: TU360 -> TU 360
    text = re.sub(r'\bTU(\d+)\b', r'TU \1', text)
    
    # "module 01" -> "MO1"
    text = re.sub(r'\bMODULE\s+0?(\d)\b', r'MO\1', text)
    
    # "installation" alone -> "PRESSURE GAUGE INSTALLATION"
    if text == 'INSTALLATION':
        text = 'PRESSURE GAUGE INSTALLATION'
    
    return text

# Apply standardization to Task_2
df['Task_2'] = df['Task_2'].apply(standardize_task2)

# Show unique values
print("\nTask_2 unique values AFTER standardization:")
for task in sorted([t for t in df['Task_2'].unique() if pd.notna(t)]):
    print(f"  - {task}")


Task_2 unique values AFTER standardization:
  - 150H INSPECTION
  - FCU REPLACEMENT
  - HMU INSTALLATION
  - MO3 7 TU 198
  - MO3 REMOVAL
  - MO3 REPLACEMENT
  - MO4 REPLACEMENT
  - MO5 REPLACEMENT
  - PRESSURE GAUGE INSTALLATION
  - REPLACEMENT FRONT FIREWALL MO1
  - SELOL SEAL CHANGE
  - STANDARD EXCHANGE 02
  - STANDARD EXCHANGE 03
  - TU 360
  - TU 377
  - UPDATE CYCLES


In [12]:
df['Task_3'].unique()

array([None, '5 REPLACEMENT', '03', '500h', 'OVSP DRAIN VALVE'],
      dtype=object)

In [13]:
def standardize_task3(text):
    """
    Standardization rules specific for Task_3 column
    """
    if pd.isna(text):
        return text
    
    # Convert to uppercase first
    text = text.upper()
    
    # "5 REPLACEMENT" -> "MO5 REPLACEMENT"
    # "3 REPLACEMENT" -> "MO3 REPLACEMENT"
    text = re.sub(r'\b(\d)\s+REPLACEMENT\b', r'MO\1 REPLACEMENT', text)
    
    # "03" or "02" alone -> "STANDARD EXCHANGE 03" or "STANDARD EXCHANGE 02"
    if re.match(r'^0\d$', text):
        text = f'STANDARD EXCHANGE {text}'
    
    # "150h", "500h", "600h" -> "150H INSPECTION", "500H INSPECTION", "600H INSPECTION"
    text = re.sub(r'\b(\d+)H\b', r'\1H INSPECTION', text)
     
    return text

# Apply standardization to Task_2
df['Task_3'] = df['Task_3'].apply(standardize_task3)

# Show unique values
print("\nTask_3 unique values AFTER standardization:")
for task in sorted([t for t in df['Task_3'].unique() if pd.notna(t)]):
    print(f"  - {task}")


Task_3 unique values AFTER standardization:
  - 500H INSPECTION
  - MO5 REPLACEMENT
  - OVSP DRAIN VALVE
  - STANDARD EXCHANGE 03


In [14]:
df['Task_4'].unique()

array([None, '600h inspection', 'TU 374'], dtype=object)

In [15]:
def standardize_task4(text):
    """
    Standardization rules specific for Task_4 column
    """
    if pd.isna(text):
        return text
    
    # Convert to uppercase first
    text = text.upper()
    
    return text

# Apply standardization to Task_2
df['Task_4'] = df['Task_4'].apply(standardize_task4)

# Show unique values
print("\nTask_4 unique values AFTER standardization:")
for task in sorted([t for t in df['Task_4'].unique() if pd.notna(t)]):
    print(f"  - {task}")


Task_4 unique values AFTER standardization:
  - 600H INSPECTION
  - TU 374


In [16]:
df[['Brief Description of Technical Intervention', 'Task_1', 'Task_2', 'Task_3', 'Task_4']].head(30)

,Brief Description of Technical Intervention,Task_1,Task_2,Task_3,Task_4
0,HMU,HMU,None,None,None
1,MODULE 4 & 5 REPLACEMENT,MO4 REPLACEMENT,MO5 REPLACEMENT,None,None
2,"TU347,MO5 REPLACEMENT",TU 347,MO5 REPLACEMENT,None,None
3,HMU SLEEVE REPLACEMENT,HMU SLEEVE REPLACEMENT,None,None,None
4,MODULE 1 REPLACEMENT,MO1 REPLACEMENT,None,None,None
5,MODULE 5 REPLACEMENT,MO5 REPLACEMENT,None,None,None
6,TU 166 APPLICATION,TU 166,None,None,None
7,TU 166 APPLICATION,TU 166,None,None,None
8,HMU TBO EXTENTION,HMU TBO EXTENTION,None,None,None
9,O RING REPLACEMENT MO5,O RING MO5 REPLACEMENT,None,None,None


In [17]:
# Create df_mod1: insert Task columns after 'Brief Description of Technical Intervention'
df_mod1 = df.copy()

# Find the index of 'Brief Description of Technical Intervention' column
col_index = df_mod1.columns.get_loc('Brief Description of Technical Intervention')

# Get column list and insert Task columns after it
cols = df_mod1.columns.tolist()
task_cols = ['Task_1', 'Task_2', 'Task_3', 'Task_4']

# Remove task columns if they already exist in other positions
for task_col in task_cols:
    if task_col in cols:
        cols.remove(task_col)

# Insert task columns after 'Brief Description of Technical Intervention'
for i, task_col in enumerate(task_cols):
    cols.insert(col_index + 1 + i, task_col)

# Reorder dataframe
df_mod1 = df_mod1[cols]
df_mod1.head(20)

,Year,Mth,Day,Engine,Variant,Operator,Brief Description of Technical Intervention,Task_1,Task_2,Task_3,Task_4,Intervention level,By
0,2013,MARCH,14,Accessories,NaN,PT TRAVIRA,HMU,HMU,None,None,None,L2,BS
1,2013,MARCH,14,ARRIEL 2,2S2,PT TRAVIRA,MODULE 4 & 5 REPLACEMENT,MO4 REPLACEMENT,MO5 REPLACEMENT,None,None,L2,BS
2,2013,MARCH,27,ARRIEL 1,1S1,MHS,"TU347,MO5 REPLACEMENT",TU 347,MO5 REPLACEMENT,None,None,L3,BS
3,2013,APR,9,Accessories,2D,PT AIRFAST,HMU SLEEVE REPLACEMENT,HMU SLEEVE REPLACEMENT,None,None,None,L2,BS
4,2013,MAY,10,ARRIEL 1,1S1,MHS,MODULE 1 REPLACEMENT,MO1 REPLACEMENT,None,None,None,L2,BS
5,2013,MAY,14,ARRIEL 1,1S1,MHS,MODULE 5 REPLACEMENT,MO5 REPLACEMENT,None,None,None,L2,KA
6,2013,MAY,19,ARRIEL 2,2C2,VNH NORTH,TU 166 APPLICATION,TU 166,None,None,None,L3,BS
7,2013,MAY,22,ARRIEL 2,2C2,VNH NORTH,TU 166 APPLICATION,TU 166,None,None,None,L3,BS
8,2013,MAY,27,Accessories,2S2,PT TRAVIRA,HMU TBO EXTENTION,HMU TBO EXTENTION,None,None,None,L2,BS
9,2013,JUNE,10,ARRIEL 1,1S1,MHS,O RING REPLACEMENT MO5,O RING MO5 REPLACEMENT,None,None,None,L2,KA


In [18]:
print(df_mod1.columns)

Index(['Year', 'Mth', 'Day', 'Engine', 'Variant', 'Operator',
       'Brief Description of Technical Intervention', 'Task_1', 'Task_2',
       'Task_3', 'Task_4', 'Intervention level', 'By'],
      dtype='object')


### Day and Duration

In [20]:
df_mod1['Day'].dropna()
df_mod1['Day'].unique()

array(['14', '27', '9', '10', '19', '22', '4', '12', '15', '18', '1', '8',
       '24', '25', '31', '5', '20', '13', '17', '21', '26', '28', '29',
       '23', '3', '16', '2', '30', '11', '25 - 29', '2 to 5', '15-20',
       '18 -19', '2 to 4', '5 to 8', '15 to 19', '20 to 22', '3 to 4',
       '5 to 6', '13 TO 19', '6', '6 TO 15', nan, '27 TO 29', '29 TO 3',
       '7', '14-15', '16-18', '23-24', '24-25', '30-01', '12-Nov',
       'Job pending', 'rejected mo2.T.A stopped', '13-Dec', '22-26',
       '2 TO 8', '26-28', '12 to 16', '19 to 23', '5 to 12', '4 to 5',
       '2 to 6', '23 to 24', '15 to 21', '18 to 20', '13 to 16',
       '12 to 17', '19 , 20', '8,9', '10,11', '13,14', '16, 17', '18, 19',
       '14,15', '20,21', '24-28', '19-20', '22-28', '5 12', '10 T0 11',
       '2&3', '22 to 23', '21 to 22', '23 TO 25', '11 to 14', '25 To 28',
       '/', '13 TO 17', '28 - 29', '30 - 31', '12 FEB- 18 MAR',
       '30 MAR -3 APR'], dtype=object)

In [21]:
values_to_remove = ['Job pending', 'rejected mo2.T.A stopped', '/']
df_mod1 = df_mod1[~df_mod1['Day'].isin(values_to_remove)]
df_mod1 = df_mod1.dropna(subset=['Day'])
df_mod1['Day'].unique()

array(['14', '27', '9', '10', '19', '22', '4', '12', '15', '18', '1', '8',
       '24', '25', '31', '5', '20', '13', '17', '21', '26', '28', '29',
       '23', '3', '16', '2', '30', '11', '25 - 29', '2 to 5', '15-20',
       '18 -19', '2 to 4', '5 to 8', '15 to 19', '20 to 22', '3 to 4',
       '5 to 6', '13 TO 19', '6', '6 TO 15', '27 TO 29', '29 TO 3', '7',
       '14-15', '16-18', '23-24', '24-25', '30-01', '12-Nov', '13-Dec',
       '22-26', '2 TO 8', '26-28', '12 to 16', '19 to 23', '5 to 12',
       '4 to 5', '2 to 6', '23 to 24', '15 to 21', '18 to 20', '13 to 16',
       '12 to 17', '19 , 20', '8,9', '10,11', '13,14', '16, 17', '18, 19',
       '14,15', '20,21', '24-28', '19-20', '22-28', '5 12', '10 T0 11',
       '2&3', '22 to 23', '21 to 22', '23 TO 25', '11 to 14', '25 To 28',
       '13 TO 17', '28 - 29', '30 - 31', '12 FEB- 18 MAR',
       '30 MAR -3 APR'], dtype=object)

In [22]:
# Convert datetime objects and handle date strings like '12-Nov', '13-Dec'
import datetime

def convert_day_value(x):
    """Convert Day values: datetime objects to day numbers, date strings to proper format"""
    # Handle datetime objects
    if isinstance(x, datetime.datetime):
        return x.day
    
    # Handle string date formats like '12-Nov' or '13-Dec'
    if isinstance(x, str) and re.match(r'^\d{1,2}-[A-Za-z]{3}$', x.strip()):
        # Extract day and month parts
        parts = x.strip().split('-')
        day_num = parts[0]
        month_abbr = parts[1].upper()
        # Return as "day MONTH" format (e.g., "12 NOV")
        return f"{day_num} {month_abbr}"
    
    return x

df_mod1['Day'] = df_mod1['Day'].apply(convert_day_value)

print("Day column values after conversion:")
print(df_mod1['Day'].unique())

Day column values after conversion:
['14' '27' '9' '10' '19' '22' '4' '12' '15' '18' '1' '8' '24' '25' '31'
 '5' '20' '13' '17' '21' '26' '28' '29' '23' '3' '16' '2' '30' '11'
 '25 - 29' '2 to 5' '15-20' '18 -19' '2 to 4' '5 to 8' '15 to 19'
 '20 to 22' '3 to 4' '5 to 6' '13 TO 19' '6' '6 TO 15' '27 TO 29'
 '29 TO 3' '7' '14-15' '16-18' '23-24' '24-25' '30-01' '12 NOV' '13 DEC'
 '22-26' '2 TO 8' '26-28' '12 to 16' '19 to 23' '5 to 12' '4 to 5'
 '2 to 6' '23 to 24' '15 to 21' '18 to 20' '13 to 16' '12 to 17' '19 , 20'
 '8,9' '10,11' '13,14' '16, 17' '18, 19' '14,15' '20,21' '24-28' '19-20'
 '22-28' '5 12' '10 T0 11' '2&3' '22 to 23' '21 to 22' '23 TO 25'
 '11 to 14' '25 To 28' '13 TO 17' '28 - 29' '30 - 31' '12 FEB- 18 MAR'
 '30 MAR -3 APR']


In [23]:
def preprocess_day_transitions(row):
    """
    Pre-process Day values to handle month/year transitions
    E.g., "30-01" in December becomes "30 DEC-01 JAN"
    E.g., "29 TO 3" becomes "29 DEC-03 JAN" (or appropriate month based on row's Mth)
    """
    day_value = str(row['Day']).strip()
    month_str = str(row['Mth']).strip().upper()
    
    # Month mapping
    month_map = {
        'JANUARY': 1, 'JAN': 1,
        'FEBRUARY': 2, 'FEB': 2,
        'MARCH': 3, 'MAR': 3,
        'APRIL': 4, 'APR': 4,
        'MAY': 5,
        'JUNE': 6, 'JUN': 6,
        'JULY': 7, 'JUL': 7,
        'AUGUST': 8, 'AUG': 8,
        'SEPTEMBER': 9, 'SEP': 9, 'SEPT': 9,
        'OCTOBER': 10, 'OCT': 10,
        'NOVEMBER': 11, 'NOV': 11,
        'DECEMBER': 12, 'DEC': 12
    }
    
    month_names = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                   'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    
    # Check if it's numeric-only with a separator (potential transition)
    # Match patterns like "30-01", "29 TO 3", "30-1", etc.
    # Use proper regex alternation with parentheses, not character class
    if re.match(r'^\d+\s*(-|,|TO|To|T0|to)\s*\d+$', day_value):
        # Split by various separators using the same pattern
        parts = re.split(r'\s*(?:-|,|TO|To|T0|to)\s*', day_value)
        if len(parts) == 2:
            start_day = int(parts[0])
            end_day = int(parts[1])
            
            # If end_day < start_day, it's likely a month transition
            if end_day < start_day:
                current_month = month_map.get(month_str, 1)
                next_month = (current_month % 12) + 1
                
                current_month_name = month_names[current_month - 1]
                next_month_name = month_names[next_month - 1]
                
                # Transform to include month names
                return f"{start_day} {current_month_name}-{end_day} {next_month_name}"
    
    return day_value

df_mod1['Day'] = df_mod1.apply(preprocess_day_transitions, axis=1)

print("Day column after preprocessing month transitions:")
print("\nChecking specific cases:")
print("  - Rows with '29 TO 3' pattern:")
print(df_mod1[df_mod1['Day'].str.contains('29.*3', na=False)][['Year', 'Mth', 'Day']].head())
print("\n  - Rows with '30-01' pattern:")
print(df_mod1[df_mod1['Day'].str.contains('30.*1', na=False)][['Year', 'Mth', 'Day']].head())

Day column after preprocessing month transitions:

Checking specific cases:
  - Rows with '29 TO 3' pattern:
     Year   Mth           Day
157  2016  July  29 JUL-3 AUG

  - Rows with '30-01' pattern:
     Year       Mth           Day
169  2016  December  30 DEC-1 JAN
309  2020   January       30 - 31


### Split days to start and end date

In [24]:
def split_day_range(day_value):
    """
    Split Day value by separators: comma, -, TO, To, T0, to, &
    Special handling for dates with month names (e.g., '12 FEB- 18 MAR')
    Returns (start_date, end_date)
    If only one value, both start and end are the same
    """
    day_str = str(day_value).strip()
    
    # Check if it contains month names (letters)
    has_letters = bool(re.search(r'[A-Za-z]', day_str))
    
    if has_letters:
        # For date ranges with months: split by comma, -, TO, To, T0, to, &
        parts = re.split(r'\s*(?:,|-|TO|To|T0|to|&)\s*', day_str)
    else:
        # For numeric-only values: split by comma, -, TO, To, T0, to, &, or space
        parts = re.split(r'\s*(?:,|-|TO|To|T0|to|&)\s*|\s+', day_str)
    
    # Filter out empty strings
    parts = [p.strip() for p in parts if p.strip()]
    
    if len(parts) == 0:
        return day_value, day_value
    elif len(parts) == 1:
        # Single value: both start and end are the same
        return parts[0], parts[0]
    else:
        # Two or more values: first is start, second is end
        return parts[0], parts[1]

# Apply the splitting function
df_mod1[['start_date', 'end_date']] = df_mod1['Day'].apply(lambda x: pd.Series(split_day_range(x)))

print("\nCreated start_date and end_date columns")
print("Sample of Day column parsing:")
print(df_mod1[['Day', 'start_date', 'end_date']].head(165))


Created start_date and end_date columns
Sample of Day column parsing:
              Day start_date end_date
0              14         14       14
1              14         14       14
2              27         27       27
3               9          9        9
4              10         10       10
..            ...        ...      ...
167         23-24         23       24
168         24-25         24       25
169  30 DEC-1 JAN     30 DEC    1 JAN
170        12 NOV     12 NOV   12 NOV
202             6          6        6

[165 rows x 3 columns]


In [25]:
df_mod1['end_date'].unique()

array(['14', '27', '9', '10', '19', '22', '4', '12', '15', '18', '1', '8',
       '24', '25', '31', '5', '20', '13', '17', '21', '26', '28', '29',
       '23', '3', '16', '2', '30', '11', '6', '3 AUG', '7', '1 JAN',
       '12 NOV', '13 DEC', '18 MAR', '3 APR'], dtype=object)

### Calculate duration

In [26]:
def calculate_duration_from_dates(row):
    """
    Calculate duration in days from start_date and end_date
    If both contain month names (e.g., "12 FEB" and "18 MAR"), use Year and Mth to resolve
    Handles year transitions (e.g., "30 DEC" to "1 JAN")
    """
    start = str(row['start_date']).strip()
    end = str(row['end_date']).strip()
    
    try:
        # Check if dates contain letters (month names)
        start_has_month = bool(re.search(r'[A-Za-z]', start))
        end_has_month = bool(re.search(r'[A-Za-z]', end))
        
        if start_has_month or end_has_month:
            # Contains month names - need Year and Mth context
            year = row['Year']
            
            # Month name mapping
            month_map = {
                'JANUARY': 1, 'JAN': 1,
                'FEBRUARY': 2, 'FEB': 2,
                'MARCH': 3, 'MAR': 3,
                'APRIL': 4, 'APR': 4,
                'MAY': 5,
                'JUNE': 6, 'JUN': 6,
                'JULY': 7, 'JUL': 7,
                'AUGUST': 8, 'AUG': 8,
                'SEPTEMBER': 9, 'SEP': 9, 'SEPT': 9,
                'OCTOBER': 10, 'OCT': 10,
                'NOVEMBER': 11, 'NOV': 11,
                'DECEMBER': 12, 'DEC': 12
            }
            
            # Parse start date
            parts_start = start.split()
            day_start = int(parts_start[0])
            month_start = month_map.get(parts_start[1].upper()) if len(parts_start) > 1 else None
            
            # Parse end date
            parts_end = end.split()
            day_end = int(parts_end[0])
            month_end = month_map.get(parts_end[1].upper()) if len(parts_end) > 1 else None
            
            if month_start and month_end:
                from datetime import datetime
                
                # Handle year transition: if end_month < start_month, increment year for end date
                year_start = year
                year_end = year + 1 if month_end < month_start else year
                
                start_datetime = datetime(year_start, month_start, day_start)
                end_datetime = datetime(year_end, month_end, day_end)
                duration = (end_datetime - start_datetime).days + 1
                return duration
            else:
                return None
        else:
            # Simple numeric case
            day_start = int(start)
            day_end = int(end)
            duration = day_end - day_start + 1
            
            # If duration is negative or zero, it's likely a month transition
            # Return None so it gets handled by preprocessing
            if duration <= 0:
                return None
            
            return duration
            
    except Exception as e:
        return None

# Apply duration calculation
df_mod1['duration'] = df_mod1.apply(calculate_duration_from_dates, axis=1)

print("\nDuration column created")
print("\nChecking for problematic durations:")
print("Rows with negative or zero duration:")
problematic = df_mod1[(df_mod1['duration'].notna()) & (df_mod1['duration'] <= 0)]
print(f"Count: {len(problematic)}")
if len(problematic) > 0:
    print(problematic[['Year', 'Mth', 'Day', 'start_date', 'end_date', 'duration']].head(20))

print("\nRows with None duration:")
none_dur = df_mod1[df_mod1['duration'].isna()]
print(f"Count: {len(none_dur)}")
if len(none_dur) > 0:
    print(none_dur[['Year', 'Mth', 'Day', 'start_date', 'end_date', 'duration']].head(20))

print("\nSample of calculated durations:")
print(df_mod1[['Day', 'start_date', 'end_date', 'duration']].head(152))


Duration column created

Checking for problematic durations:
Rows with negative or zero duration:
Count: 0

Rows with None duration:
Count: 0

Sample of calculated durations:
              Day start_date end_date  duration
0              14         14       14         1
1              14         14       14         1
2              27         27       27         1
3               9          9        9         1
4              10         10       10         1
..            ...        ...      ...       ...
154            27         27       27         1
155            28         28       28         1
156            29         29       29         1
157  29 JUL-3 AUG     29 JUL    3 AUG         6
158             5          5        5         1

[152 rows x 4 columns]


In [27]:
# Create df_mod2 with start_date, end_date, and duration columns after Day column
df_mod2 = df_mod1.copy()

# Find the index of 'Day' column
day_col_index = df_mod2.columns.get_loc('Day')

# Get column list
cols = df_mod2.columns.tolist()

# Remove start_date, end_date, duration if they exist
for col in ['start_date', 'end_date', 'duration']:
    if col in cols:
        cols.remove(col)

# Insert start_date, end_date, duration after Day column
cols.insert(day_col_index + 1, 'start_date')
cols.insert(day_col_index + 2, 'end_date')
cols.insert(day_col_index + 3, 'duration')

# Reorder dataframe
df_mod2 = df_mod2[cols]

print("df_mod2 created with start_date, end_date, and duration columns after Day")
print(df_mod2.columns.tolist())
df_mod2.head()

df_mod2 created with start_date, end_date, and duration columns after Day
['Year', 'Mth', 'Day', 'start_date', 'end_date', 'duration', 'Engine', 'Variant', 'Operator', 'Brief Description of Technical Intervention', 'Task_1', 'Task_2', 'Task_3', 'Task_4', 'Intervention level', 'By']


,Year,Mth,Day,start_date,end_date,duration,Engine,Variant,Operator,Brief Description of Technical Intervention,Task_1,Task_2,Task_3,Task_4,Intervention level,By
0,2013,MARCH,14,14,14,1,Accessories,NaN,PT TRAVIRA,HMU,HMU,None,None,None,L2,BS
1,2013,MARCH,14,14,14,1,ARRIEL 2,2S2,PT TRAVIRA,MODULE 4 & 5 REPLACEMENT,MO4 REPLACEMENT,MO5 REPLACEMENT,None,None,L2,BS
2,2013,MARCH,27,27,27,1,ARRIEL 1,1S1,MHS,"TU347,MO5 REPLACEMENT",TU 347,MO5 REPLACEMENT,None,None,L3,BS
3,2013,APR,9,9,9,1,Accessories,2D,PT AIRFAST,HMU SLEEVE REPLACEMENT,HMU SLEEVE REPLACEMENT,None,None,None,L2,BS
4,2013,MAY,10,10,10,1,ARRIEL 1,1S1,MHS,MODULE 1 REPLACEMENT,MO1 REPLACEMENT,None,None,None,L2,BS


In [28]:
df_mod2['duration'].isna().sum()

np.int64(0)

In [29]:
# Calculate task count and task duration
import math

def count_tasks(row):
    """Count non-null tasks in Task_1 through Task_4"""
    task_cols = ['Task_1', 'Task_2', 'Task_3', 'Task_4']
    return sum(1 for col in task_cols if pd.notna(row[col]))

def calculate_task_duration(row):
    """
    Calculate duration per task by dividing total duration by task count.
    Round up with minimum of 1 day.
    """
    if pd.isna(row['duration']) or row['duration'] == 0:
        return None
    
    task_count = row['task_count']
    if task_count == 0:
        return None
    
    # Divide duration by task count and round up
    task_duration = math.ceil(row['duration'] / task_count)
    
    # Ensure minimum of 1 day
    return max(1, task_duration)

# Add task_count column
df_mod2['task_count'] = df_mod2.apply(count_tasks, axis=1)

# Add task_duration column
df_mod2['task_duration'] = df_mod2.apply(calculate_task_duration, axis=1)

print("Task count and task duration calculated!")
print("\nSample calculations:")
print(df_mod2[['Day', 'duration', 'Task_1', 'Task_2', 'Task_3', 'Task_4', 'task_count', 'task_duration']].head(30))

task_col_index = df_mod2.columns.get_loc('Task_4')

for col in ['task_count', 'task_duration']:
    if col in cols:
        cols.remove(col)

cols.insert(task_col_index + 1, 'task_count')
cols.insert(task_col_index + 2, 'task_duration')

# Reorder dataframe
df_mod2 = df_mod2[cols]

print("df_mod2 created with task_count and task_duration columns after Task_4")
print(df_mod2.columns.tolist())

Task count and task duration calculated!

Sample calculations:
   Day  duration                       Task_1           Task_2  \
0   14         1                          HMU             None   
1   14         1              MO4 REPLACEMENT  MO5 REPLACEMENT   
2   27         1                       TU 347  MO5 REPLACEMENT   
3    9         1       HMU SLEEVE REPLACEMENT             None   
4   10         1              MO1 REPLACEMENT             None   
5   14         1              MO5 REPLACEMENT             None   
6   19         1                       TU 166             None   
7   22         1                       TU 166             None   
8   27         1            HMU TBO EXTENTION             None   
9   10         1       O RING MO5 REPLACEMENT             None   
10  14         1                       TU 166             None   
11   4         1              MO1 REPLACEMENT  MO5 REPLACEMENT   
12  12         1       HMU SLEEVE REPLACEMENT             None   
13  15       

In [30]:
# Standardize Intervention level to L1, L2, L3 format
def standardize_intervention_level(level):
    """Standardize intervention level to L1, L2, or L3 format"""
    if pd.isna(level):
        return level
    
    level_str = str(level).strip().upper()
    
    # Extract the number from various formats
    # Match patterns like: L1, L2, L3, Level 1, Level 2, etc.
    match = re.search(r'[L]?(\d)', level_str)
    if match:
        number = match.group(1)
        return f'L{number}'
    
    return level

df_mod2['Intervention level'] = df_mod2['Intervention level'].apply(standardize_intervention_level)

print("Intervention level standardized!")
print("\nUnique intervention levels:")
print(df_mod2['Intervention level'].unique())
print("\nIntervention level distribution:")
print(df_mod2['Intervention level'].value_counts())

Intervention level standardized!

Unique intervention levels:
['L2' 'L3' 'L1' nan]

Intervention level distribution:
Intervention level
L1    181
L2     65
L3     13
Name: count, dtype: int64


In [31]:
df_mod2.dropna(subset=['Intervention level'], inplace=True)
df_mod2['Intervention level'].isna().sum()

np.int64(0)

In [32]:
df_mod2['Engine'].unique()

array(['Accessories', 'ARRIEL 2', 'ARRIEL 1', 'ARRIUS 1', 'ARRIUS 2',
       'MAKILA 2', 'ARRIEl 1', 'ARRIEL', 'ARRIUS', 'ARRIUS ', 'Arriel 2',
       'MAKILA 1', 'ARRIEL2', 'ARRRIEL 2', 'E-APU', 'MAKILA ', 'eAPU',
       'Arriel 1'], dtype=object)

In [33]:
# Standardize Engine names
def standardize_engine(engine):
    """Standardize engine names to consistent format"""
    if pd.isna(engine):
        return engine
    
    engine_str = str(engine).strip().upper()
    
    # Remove extra spaces
    engine_str = re.sub(r'\s+', ' ', engine_str)
    
    # Fix common typos and variations
    # ARRIEL variations (including typos like ARRRIEL, ARRIEl, etc.)
    if re.search(r'AR+I+E*L', engine_str):
        # Extract number if present
        if '1' in engine_str:
            return 'ARRIEL 1'
        elif '2' in engine_str:
            return 'ARRIEL 2'
        else:
            return 'ARRIEL'
    
    # ARRIUS variations
    if 'ARRIUS' in engine_str or 'ARIUS' in engine_str:
        if '1' in engine_str:
            return 'ARRIUS 1'
        elif '2' in engine_str:
            return 'ARRIUS 2'
        else:
            return 'ARRIUS'
    
    # MAKILA variations
    if 'MAKILA' in engine_str:
        if '1' in engine_str:
            return 'MAKILA 1'
        elif '2' in engine_str:
            return 'MAKILA 2'
        else:
            return 'MAKILA'
    
    # E-APU / eAPU variations
    if 'APU' in engine_str or 'E-APU' in engine_str or 'EAPU' in engine_str:
        return 'E-APU'
    
    # ACCESSORIES
    if 'ACCESSORIES' in engine_str or 'ACCESSORY' in engine_str:
        return 'ACCESSORIES'
    
    return engine_str

df_mod2['Engine'] = df_mod2['Engine'].apply(standardize_engine)

In [34]:
df_mod2['Engine'].unique()

array(['ACCESSORIES', 'ARRIEL 2', 'ARRIEL 1', 'ARRIUS 1', 'ARRIUS 2',
       'MAKILA 2', 'ARRIEL', 'ARRIUS', 'MAKILA 1', 'E-APU', 'MAKILA'],
      dtype=object)

In [35]:
df_mod2['Variant'].unique()

array([nan, '2S2', '1S1', '2D', '2C2', '2s2', '1D1', '2S1', '2C1', '1E2',
       '2C', '1A', '2B1', '1B', '2B', '1C2', '1D', '2F', '2K2', 'A1',
       'C2', 'S1', '2A1', 'E2', 'D1', 'B', 'D', 'A', 'S2', 'C1', 'F',
       'B1', 'K2', '60', 'B2', '1A1', '1A2', 'C', 'R'], dtype=object)

In [36]:
# Standardize Variant to uppercase
df_mod2['Variant'] = df_mod2['Variant'].apply(lambda x: str(x).upper() if pd.notna(x) else x)

df_mod2['Variant'].unique()

array([nan, '2S2', '1S1', '2D', '2C2', '1D1', '2S1', '2C1', '1E2', '2C',
       '1A', '2B1', '1B', '2B', '1C2', '1D', '2F', '2K2', 'A1', 'C2',
       'S1', '2A1', 'E2', 'D1', 'B', 'D', 'A', 'S2', 'C1', 'F', 'B1',
       'K2', '60', 'B2', '1A1', '1A2', 'C', 'R'], dtype=object)

In [37]:
df_mod2['Operator'].unique()

array(['PT TRAVIRA', 'MHS', 'PT AIRFAST', 'VNH NORTH', 'POOL', 'NGCP',
       'HONGIK', 'TADECO', 'AEROPEACE', 'DEAJIN AIR', 'TMUSA',
       'HELIKOREA', 'TMF POOL', 'TAS', 'SFS', 'PT EASTINDO', 'PELITA AIR',
       'MINEBEA', 'NASC', 'POOL TMF', 'HELISTAR', 'FALCOR',
       'RII BUILDERS', 'EHSI', 'COPTER', 'ARANETA', 'ICTSI', 'LGC',
       'NORTHERN RESORT', 'TANDUAY DISTILLERS', 'INAEC', 'STA LUCIA',
       'BENGUET', 'TDG', 'ALPHALAND', 'CHALLENGER AERO', 'S&R', 'PGA',
       'WORLD AVIATION', 'HELITREND', 'MACROASIA', 'GLORY INTERNATIONAL',
       'SUBIC AIR', 'HELISTAR CAMBODIA', 'MOAC', 'MNRE', 'EGAT',
       'ADVANCE AVIATION', 'S&R ', 'PT PAS', 'FORESTRY AVIATION',
       'DAEGU FIRE', 'SN AIR', 'DAEJIN AIR', 'HONGIK AIR',
       'PT MARTA BUANA', 'M.LHUILLIER', 'HAMMOCK', 'WESTAR', 'RMN',
       'PLUS', 'HEVILIFT', 'PT NUH', 'PUTD', 'INDOSTAR', 'AWAN', 'TAP',
       'WESTSTAR', 'GADING ', 'ZETRO', 'RMP', 'HEVILIFT INDONESIA',
       'GADING', 'SHE TARNOS', 'SAZMA', 'MM GOLDEN

In [38]:
df_mod2['By'].unique()

array(['BS', 'KA', 'MM', ' BS', 'Ray (TAP)', 'Jay (TAP) and Jeff',
       'Otaka(TMJ) and Ejam(GTA)', 'SIEW', 'JASMALIZAM', 'ALI', 'Jerome',
       'Jasmalizam', 'AZAHARI', 'Ali', 'Jerome METCHEDE'], dtype=object)

In [39]:
# Standardize By column
def standardize_by(by_value):
    """Standardize By column - uppercase and extract first name"""
    if pd.isna(by_value):
        return by_value
    
    # Convert to uppercase and trim
    by_str = str(by_value).strip().upper()
    
    # For entries with "AND", keep as is (multi-person entries)
    if 'AND' in by_str:
        return by_str
    
    # Extract first word (first name or department code)
    # This handles: "JEROME METCHEDE" → "JEROME", "ALI" → "ALI", "BS" → "BS"
    # Also handles: "RAY (TAP)" → "RAY"
    first_word = by_str.split()[0]
    
    return first_word

df_mod2['By'] = df_mod2['By'].apply(standardize_by)
df_mod2['By'].unique()

array(['BS', 'KA', 'MM', 'RAY', 'JAY (TAP) AND JEFF',
       'OTAKA(TMJ) AND EJAM(GTA)', 'SIEW', 'JASMALIZAM', 'ALI', 'JEROME',
       'AZAHARI'], dtype=object)

In [40]:
# Save df_mod2 to Excel and CSV
df_mod2.to_csv('Technical_Assistance_Cleaned.csv', index=False)

print("\nData saved successfully!")
print("- CSV file: Technical_Assistance_Cleaned.csv")
print(f"Total rows: {len(df_mod2)}")
print(f"Total columns: {len(df_mod2.columns)}")


Data saved successfully!
- CSV file: Technical_Assistance_Cleaned.csv
Total rows: 259
Total columns: 18
